# 初学RPM

已有的源码包含代码、链接库、宏定义等，并已通过Eclipse CDT创建了可运行的C++工程。  
想为现有的源码创建相应的rpm安装包。将学习内容记录如下。

---

学习思路：
1. 了解RPM基本知识。
2. 明确创建rpm安装包需要准备的内容。
3. 按照教程的步骤实现一个简单的demo。
4. 根据已有的工程，创建对应的rpm包。  

我作的测试安装包在`/RPM-ExerciseCode/rpmbuild`下。  
主要的学习文件是`/Books/rpm-packaging-guide.pdf`

## 1.RPM介绍

RPM Package Manager（RPM）是一个运行在RedHat，Centos，Fedora等linux系统上的包管理系统。

---

它的主要功能：
* 安装、重装、卸载、更新和认证软件安装包  
使用标准的包管理工具对软件进行安装、重装、卸载、更新和认证软件安装包。
* 使用数据库存储现有的软件安装包，用户可进行请求、认证
* 使用元数据描述安装包  
每个RPM安装包包含描述该安装包信息的元数据，比如：安装包的组件、版本、发布信息、大小、项目的URL、安装说明等信息
* 将软件初始的代码打包成源码或二进制包
* 把安装包添加进Yum仓库
* 未安装包添加数字签名，以供使用者认证  

## 2.为打包准备的源码和软件

这里的`打包`即把源码打包成RPM安装包。如果程序比较简单且依赖文件较少，可以直接使用Linux系统内核提供的`install`命令直接将程序安装到本地机器中。

---


### 2.1RPM包需要的源码  
源码可以是多种编程语言。这里以C语言为例，写一个简单的helloworld程序。
```C++
#include <stdio.h>
int main(void){
    printf("hello world \n");
    return 0;
}
```
### 2.2程序构建的方式（How Programs Are Made）
程序的源码是程序员使用程序语言写成的，要将其转变与易于机器识别的代码，这个过程会有很多方法，简化下来共有三类：
1. 本地编译程序
2. the program is interpreted by `raw interpreting`
3. the program is interpreted by `byte compiling`

#### 2.2.1本地编译代码  
使用本地的编译器编译源码，生成二进制可执行文件，可以运行在本地机器上。但是这种方法有局限：如果是64位的AMD处理器编译出的可执行文件，不能再32位的AMD机器上运行。这种安装包需要在包名上标注清楚安装的条件（32位/64位机器），即the resulting package will have architecture specified in its name。

#### 2.2.2解释型代码
一些语言是解释型语言，不需要编译成机器码，如python、bash。这种语言一句一句地执行，需要一个语言解释器（Language Interpreter）或者语言虚拟机（Language Virtual Machine）。因此用解释型语言写的软件不需要标明安装的条件（architecture -specific），但是需要在RPM包名上添加`noarch`。
解释型语言的程序既是`raw-interpreting`，也是`byte compiling`。这两种程序不同在于**程序的build过程**和**packaging步骤**。

> **raw-interpreting**类型编程语言写的程序不需要编译，直接通过解释器执行。  
> **byte-compiled**类型编程语言需要被编译成二进制代码，之后被语言虚拟机（Language Virtual Machine）执行，例如java中，先将程序编译成字节码（二进制码），再运行在JVM中。  
> **注：**一些语言可以选择以上两种方式之一进行构建程序。

### 2.3通过源码构建软件（Building Software from Source）  
|程序语言类型|执行方式|
|:--|:--|
|编译型语言|经过编译生成机器码（二进制码），链接之后再生成可执行文件，程序员可以通过执行这个可执行文件来运行程序|
|raw interpreted languages|源码直接被执行|
|byte-compiled languages|源码经过编译成二进制码后，在语言虚拟机中运行|  

#### 2.3.1本地编译代码
以上面的C++程序为例。
* 通过GCC命令手动编译，生成可执行程序。  
     -g：产生调试信息。  
     -o cello：生成文件cello。（默认为生成a.out文件）

    ```shell
    gcc -g -o cello cello.c
    ```
* 执行这个可执行程序  
    ```shell
     ./cello    
    ```  
#### 2.3.2自动构建代码  
在大规模软件开发中，使用`Makefile`和`GUN make命令`代替手工编译程序。  
* 在cello.c文件的同一路径下创建Makefile文件  

    ```shell
    # makefile document
    cello:
        gcc -g -o cello cello.c

    clean:
        rm cello  
    ```  
* 使用`make`命令，运行Makefile中cello后的命令，即编译cello.c文件。  
  clean后的命令，可以通过`make clean`命令清除已经make生成的文件。  
    ```shell
     make
    ```  
* 执行make后生成的可执行文件

    ```shell
      ./cello
    ```  

### 2.4为软件打补丁（Patching Software）  
`diff`工具被应用于为程序打补丁。

### 2.5安装软件（Installing Arbitrary Artifacts）  
通过rpm包安装的软件也要遵守Linux系统中的FHS（Filesystem Hierarchy Standard）。例如，一个可执行文件需要将其添加进系统的PATH变量中。通过rpm包向系统安装的软件是多样化的，可以是脚本（script），从源码编译的二进制代码，预处理的二进制代码等等。  
这里介绍了`install`和`make install`两种安装方式。

#### 2.5.1使用`install`命令安装  
如果程序很简单并且不需要其他的依赖文件，则可以使用`install`命令安装，这个命令由操作系统内核提供。通过`install`命令，可以将软件安装至特定的路径下，并且有特定的权限需求（with a specified set of permissions）。  
下面的示例之前的hello world代码通过`install`命令安装到`/usr/bin`路径下。安装命令如下：
```shell
    sudo install -m 0755 bello /usr/bin/bello
```  
通过命令安装后，`bello`的路径将被写入PATH环境变量中，之后可以在任何地方执行`bello`。  
>但是我不知道什么命令可以把用install安装的bello删掉  
>
>书上说，会将bello的路径写道PATH环境变量中，但其实并没有，PATH中有/usr/bin的路径，应该bash检测到输入bello命令后，是查找这个路径下是否有可执行程序bello。  
> 
>因此我删除了/usr/bin下的bello文件，应该就删除了bello，因为bello没有其他的依赖文件，不用专门删除。

#### 2.5.2使用`make install`命令安装  
`make install`命令需要通过Makefile文件来指明如果安装软件。一般，Makefile文件会由程序的开发者编写。`cello`程序的Makefile文件如下：  
   
   ```shell
    cello:
        gcc -g -o cello cello.c
    clean:
        rm cello
    install:
        #mkdir -p：参数的意思是要创建的目录
        mkdir -p $(DESTDIR)/usr/bin  
        
        #-m：参数设置文件的权限
        install -m 0755 cello $(DESTDIR)/usr/bin/cello
    ```  
根据上面的Makefile文件，可以build程序，并将其安装到本地机器中。使用命令：  
    
```shell
        make
        sudo make install
    ```    
    
### 2.6为打包准备源码  
软件开发者通常会将整个程序的全部代码整理成压缩的代码包（compressed archives of source code），然后对整个代码包进行打包成rpm包。
软件需要有一个软件证书（Software Liscense）才可以被发行。将源码打包成RPM安装包需要这些证书文件。  
>（我不太理解，要这个证书的作用，是开源社区的规定吗？）  

### 2.7将源码文件放入压缩包  
`cello`工程是hello world工程的C语言版本，这个工程只包含一个`cello.c`文件和`Makefile`文件，因此将这两个文件和`LISCENSE`文件共同压缩成`tar.gz`格式的文件。并命名这个工程为`1.0`版本。具体步骤如下：  
> 1. 将文件放入一个目录下  
```shell
    mkdir /tmp/cello-1.0  
    mv ~/cello.c /tmp/cello-1.0  
    mv ~/Makefile /tmp/cello-1.0  
    cp /tmp/LISCENSE /tmp/cello-1.0
```
> 2. 生成压缩文件，并将其拷贝到~/rpmbuild/SOURCES/目录下    
```shell
    cd /tmp/
    tar -cvzf cello-1.0.tar.gz cello-1.0
    mv /tmp/cello-1.0.tar.gz ~/rpmbuild/SOURCES/
```  
> 3. 将补丁文件加入SOURCES路径中  

综上，已经将生成RPM包的源码准备完毕。

## 3.对程序进行打包  
使用RPM作为安装包的系统是Red Hat家族的Linux操作系统：  
* Fedora  
* Centos  
* Red Hat Enterprise Linux(RHEL)  

>创建的RPM包的目录是在根目录～/rpmbuild下，可以使用tree命令，查看当前目录下的文件树的现状。

---

### 3.1RPM包  
主要有两类RPM包：  
1. 源码RPM（SRPM）
2. 二进制RPM  

SPRM包含了源码、补丁包、SPEC文件（描述了如何将源码构建称二进制的RPM）  

#### 3.1.1RPM打包工具
使用如下命令查看RPM的打包工具：
```shell
rpm -ql rpmdevtools | grep bin
```

#### 3.1.2RPM打包Workspace
```shell
#此命令在用户的根目录，即～/下创建RPM打包的工作空间：rpmbuild目录
rpmdev-setuptree

#使用tree命令查看工作空间
tree ~/rpmbuild/
```
>rpmdev-setuptree命令默认将再当前用户主目录下创建一个RPM构建根目录结构，如果需要改变次默认位置，可以修改配置文件:~/.rpmmacros中变量_topdir对应的值即可。

～/rpmbuild/路径下的文件目录为：  
`BUILD`  `RPMS`  `SOURCES`  `SPECS`  `SRPMS`  
作用如下：  

|目录|作用|
|:--|:--|
|BUILD|%buildroot目录会在这个文件夹里生成|
|BUILDROOT||
|RPMS|二进制的RPM包在这里生成，例如以x86-64/norach结尾的文件|
|SOURCES|源码和补丁文件放在这里，rpmbuild命令会在这个文件夹下寻找文件|
|SPECS|SPEC文件放在这里|
|SRPMS|若rpmbuild命令创建SRPM，则在这里生成SRPM文件|  

#### 3.1.3SPEC文件  
SEPC文件可以理解为使用rpmbuild命令构建RPM包时用的菜谱，它通过定义一些说明来告诉系统如何构建rpm包。SPEC文件前面，定义了一些元数据，之后SPEC会使用这些元数据。  
.spec文件中的宏，可以通过rpm --eval命令进行查询：  
```shell
#查_bindir宏定义的含义，输出：/usr/bin/
rpm --eval %{_bindir}
```
SPEC文件的头部：  

|元数据|说明|
|:--|:--|
|Name|The base name of the package, which should match the SPEC file name.|
|Version|The upstream version number of the software.|
|Release|The number of times this version of the software was released.Normally, set the initial value to 1%{?dist}, and increment it with each new release of the package. Reset to 1 when a new `Version` of the software is built.|
|Summary|A brief, one-line summary of the package.|
|License|The license of the software being packaged. For packages distributed in community distributions such as `Fedora` this must be an open source license abiding by the specific distribution’s licensing
guidelines.|
|URL|The full URL for more information about the program. Most often this is the upstream project website for the software being packaged.|
|Source0|Path or URL to the compressed archive of the upstream source code (unpatched, patches are handled elsewhere). This should point to an accessible and reliable storage of the archive, for example, the upstream page and not the packager’s local storage. If needed, more SourceX directives can be added, incrementing the number each time, for example: Source1, Source2, Source3, and so on.|
|Patch0|The name of the first patch to apply to the source code if necessary.If needed, more PatchX directives can be added, incrementing the number each time, for example: Patch1, Patch2, Patch3, and so on.|
|BuildArch|If the package is not architecture dependent, for example, if written entirely in an interpreted programming language, set this to `BuildArch: noarch`. If not set, the package automatically inherits the Architecture of the machine on which it is built, for example `x86_64`.|
|BuildRequires|A comma- or whitespace-separated list of packages required for building the program written in a compiled language. There can be multiple entries of `BuildRequires`, each on its own line in the SPEC file.|
|Requires|A comma- or whitespace-separated list of packages required by the software to run once installed. There can be multiple entries of `Requires`, each on its own line in the SPEC file.|
|ExcludeArch|If a piece of software can not operate on a specific processor architecture, you can exclude that architecture here.|

`Name` `Version` `Release`共同组成RPM包的名字，一般的格式为**N-V-R**。
Linux中使用rpm命令可以查询一个软件的全名：  
```shell
rpm -q python
python-2.7.5-34.el7.x86_64
```  


SPEC文件的主体：  
文件的主体（body）部分主要是通过命令是源码转换为二进制程序，然后安装。

|数据|说明|
|:--|:--|
|%description|A full description of the software packaged in the RPM. This description can span multiple lines and can be broken into paragraphs.|
|%prep|Command or series of commands to prepare the software to be built, for example, unpacking the archive in `Source0`. This directive can contain a shell script.|
|%build|Command or series of commands for actually building the software into machine code (for compiled languages) or byte code (for some interpreted languages).
|%install|Command or series of commands for copying the desired build artifacts from the `%builddir` (where the build happens) to the `%buildroot`  directory (which contains the directory structure with the files to be packaged). This usually means copying files from `~/rpmbuild/BUILD` to `~/rpmbuild/BUILDROOT` and creating the necessary directories in `~/rpmbuild/BUILDROOT`. This is only run when creating a package, not when the end-user installs the package. See Working with SPEC files for details.|
|%check|Command or series of commands to test the software. This normally includesthings such as unit tests.|
|\%files|The list of files that will be installed in the end user’s system.|
|\%changelog|A record of changes that have happened to the package between different `Version` or `Release` builds|  


#### 3.1.4BuildRoots 
>我的理解：当RPM被安装时，BuildRoot中的文件将会被安装到root相应的目录下。  

#### 3.1.5RPM Macros  
RPM宏，可以让RPM进行自动的文本替换。  
比如定义%{version}，之后使用这个宏定义作为版本号，修改版本号时只需要修改宏定义即可。  

#### 3.1.6使用SPEC文件  
将软件打包称RPM包的主要一部分就是写SPEC文件。  

使用`rpmdev-newspec`工具，创建一个新的SPEC模板文件。
在rpmbuild/SPEC/路径下使用命令：  
```shell
rpmdev-newspec cello
```

修改cellp.spec文件，按照文件中的头部、主体信息，填写。  
一定要按照规范书写.spec文件，之后的rpmbuild命令完全按照.spec文件的内容一步一步执行，如果.spec文件有误，可能会生成错误的rpm包，从而导致安装失败。 
>注意.spec文件中%file设置文件的权限，之前没有设置，导致安装的/usr/bin/cello命令权限为000，不能正常运行。  

我的.spec文件如下：  
```spec
Name:           cello
Version:        1.0
Release:        1%{?dist}
Summary:        This is a test for rpm by fxf

License:        GPLv3+
URL:            https://www.example.com/%{name}
Source0:        https://www.example.com/%{name}/releases/%{name}-%{version}.tar.gz

BuildRequires:  gcc
BuildRequires:  make

%description
This is the first test about cello and RPM package making by fxf

%prep
%setup -q   #提取源码到 BUILD 目录; -q 指不显示输出（quietly）


%build
make %{?_smp_mflags}


%install
#rm -rf $RPM_BUILD_ROOT
%make_install


%files
%license LICENSE
%{_bindir}/%{name}
%attr(0755,root,root) %{_bindir}/%{name}


%changelog
* Wed Jan 16 2019 fxf <735557668@qq.com> - 1.0
- First cello package
                             
```


### 3.2创建RPM包  
创建RPM包分为两类：  
1. 创建Source RPM
2. 创建Binary RPM  

**创建Source RPM（SRPM）**  
使用命令：  
```shell
#SRPM
rpmbuild -bs cello.spec
```

> 此命令会在～/rpmbuild/SRPMS目录下生成cello-1.0-1.el7.src.rpm文件，这个文件不能直接安装，只是包含了对rpm的源码压缩包.tar.gz和编译说明文件.spec文件，并没有二进制的可执行程序来安装。

**创建Binary RPM**  
```shell
#根据SRPM生成的rpm重新编译生成Source RPM
rpmbuild --rebuild ~/rpmbuild/SRPMS/cello-1.0-1.el7.src.rpm

#直接生成Source RPM
rpmbuild -bb ~/rpmbuild/SPECS/cello.spec
```
> 此命令会在路径~/rpmbuild/RPMS路径下生成rpm的安装包，检验正确性后可以直接安装，成功运行。

### 3.3检查SPEC，RPM包是否正确##  
使用rpmlink命令  
```shell
#检查SPEC文件
rpmlint ~/rpmbuild/SPECS/cello.spec

#检查rpm包
rpmlint ~/rpmbuild/SRPMS/cello-1.0-1.el7.src.rpm
rpmlint ~/rpmbuild/RPMS/x86_64/cello-1.0-1.el7.x86_64.rpm
```

